In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 사용할 한글 폰트 설정 및 적용
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

class DataLoader:
    """단일 책임 원칙 (SRP): 데이터 로딩과 초기 처리"""
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load_data(self) -> pd.DataFrame:
        return pd.read_csv(self.file_path, encoding="cp949")

class DataCleaner:
    """단일 책임 원칙 (SRP): 데이터 클리닝"""
    @staticmethod
    def nan_filter(df: pd.DataFrame) -> pd.DataFrame:
        df = df.replace(r'\N', np.nan)
        df = df.replace(r'\\N', np.nan)
        df = df.fillna('ETC')
        return df

    @staticmethod
    def clean_data(df: pd.DataFrame) -> pd.DataFrame:
        df = DataCleaner.nan_filter(df)
        df.drop(["대여소번호", "운동량", "탄소량"], axis=1, inplace=True)
        df.columns = ["date(Month)", "addr", "code", "Gender", "Birth", "Usecount", "Use_distant(m)", "UseTime(m)"]
        df['Gender'] = df['Gender'].apply(str.upper)
        return df

class DataProcessor:
    """단일 책임 원칙 (SRP): 데이터 처리 및 통합"""
    @staticmethod
    def concat_data(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
        return pd.concat([df1, df2], axis=0)

class Plotter:
    """단일 책임 원칙 (SRP): 데이터 시각화"""
    @staticmethod
    def plot_pie_chart(df: pd.DataFrame):
        Birth_gro = df.groupby("Birth").agg({"Usecount": "sum"})
        labels = ["10대", "20대", "30대", "40대", "50대", "60대", "70대 이상", "기타"]
        explode = [0.1, 0.1, 0, 0, 0, 0, 0, 0]
        colors = ['#ff9999', 'cornflowerblue', 'gold', 'orchid', 'green', "#77BFE2", "#d395d0", "#8fd9b6"]
        
        plt.figure(figsize=(15, 10))
        plt.pie(Birth_gro["Usecount"], labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, explode=explode, shadow=True, colors=colors, textprops={'fontsize': 14})
        plt.title("2023년도 나이별 따릉이 사용률", fontsize=20)
        plt.show()

    @staticmethod
    def plot_heatmap(df: pd.DataFrame):
        sns.heatmap(df.corr(numeric_only=True), annot=True, fmt='.2f', linewidths=5)
        plt.title("23년 월별 이용 상관관계")
        plt.show()

    @staticmethod
    def plot_scatter(df: pd.DataFrame):
        color = {"M": "blue", "F": "orange", "ETC": "green"}
        sns.scatterplot(data=df, x='Usecount', y='Birth', hue="Gender", hue_order=("M", "F", "ETC"), palette=color)
        plt.ylabel("age")
        plt.rc('font', size=20)
        plt.title("23년 성별에 따른 나이대별 이용 횟수")
        plt.show()

class BikeUsageAnalysis:
    """의존 역전 원칙 (DIP): 고수준 모듈"""
    def __init__(self, loader1: DataLoader, loader2: DataLoader, cleaner: DataCleaner, processor: DataProcessor, plotter: Plotter):
        self.loader1 = loader1
        self.loader2 = loader2
        self.cleaner = cleaner
        self.processor = processor
        self.plotter = plotter

    def run_analysis(self):
        data_1 = self.loader1.load_data()
        data_2 = self.loader2.load_data()
        
        clean_data_1 = self.cleaner.clean_data(data_1)
        clean_data_2 = self.cleaner.clean_data(data_2)
        
        all_month = self.processor.concat_data(clean_data_1, clean_data_2)
        
        self.plotter.plot_pie_chart(all_month)
        self.plotter.plot_heatmap(all_month)
        self.plotter.plot_scatter(all_month)

# 인스턴스 생성 및 메서드 호출
loader1 = DataLoader("data/1_6_M.csv")
loader2 = DataLoader("data/7_12_M.csv")
cleaner = DataCleaner()
processor = DataProcessor()
plotter = Plotter()

analysis = BikeUsageAnalysis(loader1, loader2, cleaner, processor, plotter)
analysis.run_analysis()
